In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os 

## 1.Get all links for zimuku items

In [2]:
## send the request
url = "http://www.zimuku.net/search?q=&t=onlyst"
result = requests.get(url)
print(result.status_code)

200


In [3]:
## get page number
c = result.content
soup = BeautifulSoup(c,"lxml")
pages =  soup.find('div',{'class':'pagination r clearfix'}).find('div')
page_num = pages.findAll('a')[-1].getText()
page_num = int(page_num)
print(page_num)


12993


In [4]:
## get download link from title link 
def get_down_link(url):
    #url = "http://www.zimuku.net/shooter/57265.html"
    try:
        result = requests.get(url)
        if result.status_code != 200:
            return None
        c = result.content
        soup = BeautifulSoup(c,"lxml")
        down_link =  soup.find('li',{'class':'li dlsub'}).find('a').get('href')
        return down_link
    except:
        return None

In [5]:
## download 
def download_zip(url,folder):
    try:
        result = requests.get(url)
        fname = result.headers['Content-Disposition'].split('"')[1]
        res= result.content
        with open( folder + fname, "wb" ) as f :
                f.write( res )
        return True
    except:
        return False

### set up some global variables

In [6]:
## create doc_list to hold all links 
doc_list = {'title':[],'link':[],'status':[]}
page_start = 12932
page_end = page_num                              ## process 3, 6303 to 10000 pages
folder = 'data/process4/data_download/'

# create folder if folder does not exist
if not os.path.exists(folder):
    os.makedirs(folder)        


In [7]:

## loop through each page
for p in range(page_start,page_end):
    print(p-1, 'success')
    try:
        page_url = url + '&p=' + str(p)
        result = requests.get(page_url)
        #print(result.status_code)
        c = result.content
        soup = BeautifulSoup(c,"lxml")
        docs = soup.find('div',{'class':'box clearfix'}).findAll('div',{'class':'persub clearfix'})

        for doc in docs:

            ## get title 
            link_tag = doc.find('h1',{'class':'title'}).find('a')
            title = link_tag.getText().strip(' \t\n\r@').replace('\n','').replace('\r',' ')
            if title == '_':
                continue
            elif "/" in title:
                 title = title.split("/")[0]

            link='http://www.zimuku.net'+link_tag.get('href')

            ## get download link 
            down_link = get_down_link(link)

            doc_list['title'].append(title)
            doc_list['link'].append(down_link)
        
            ## download file 
            try:
                response = download_zip(down_link,folder)
                doc_list['status'].append(response)
            except:
                doc_list['status'].append(False)
    except:
        continue

print(len(doc_list['link']))

12931 success
12932 success
12933 success
12934 success
12935 success
12936 success
12937 success
12938 success
12939 success
12940 success
12941 success
12942 success
12943 success
12944 success
12945 success
12946 success
12947 success
12948 success
12949 success
12950 success
12951 success
12952 success
12953 success
12954 success
12955 success
12956 success
12957 success
12958 success
12959 success
12960 success
12961 success
12962 success
12963 success
12964 success
12965 success
12966 success
12967 success
12968 success
12969 success
12970 success
12971 success
12972 success
12973 success
12974 success
12975 success
12976 success
12977 success
12978 success
12979 success
12980 success
12981 success
12982 success
12983 success
12984 success
12985 success
12986 success
12987 success
12988 success
12989 success
12990 success
12991 success
1220


In [8]:
## export links file 
df = pd.DataFrame(doc_list)
df.to_csv('zimuku_links_v1.csv')
df.ix[0:5]

,link,status,title
0,http://shooter.zimuku.net/download/OTI4fEVyaW5...,True,Erin_Brockovich_EnCn_aNBc_Sub_Cslrj_JDJ_D9
1,http://shooter.zimuku.net/download/OTI3fGdob3N...,True,Ghost in Love
2,http://shooter.zimuku.net/download/OTI2fDAwN19...,True,007铁金刚大战特务飞龙
3,http://shooter.zimuku.net/download/OTMzfGEuY2x...,True,燃眉追击
4,http://shooter.zimuku.net/download/MzkwMXxhX2N...,True,a_clear_and_present_danger_1994_ac3_2cd
5,http://shooter.zimuku.net/download/OTIxfEJsYWN...,True,黑鹰计划


## Now start downloading